In [1]:
import polars as pl

In [2]:
file_path = "output/per_move.parquet"
fraction = 0.1
seed = 42
df = pl.scan_parquet(file_path)
df.head().collect()

id,move_idx,move_uci,white_elo,black_elo,avg_elo,state0,state1,state2,state3,state4,state5,state6,state7
i64,i64,str,i64,i64,f64,u32,u32,u32,u32,u32,u32,u32,u32
1,1,"""e2e4""",1227,1190,1208.5,1110856484,286331153,0,0,0,0,2004318071,2828843402
1,2,"""e7e5""",1227,1190,1208.5,1110856484,286265617,0,65536,0,0,2004318071,2828843402
1,3,"""f1c4""",1227,1190,1208.5,1110856484,286265617,0,65536,458752,0,2003859319,2828843402
1,4,"""g8f6""",1227,1190,1208.5,1107710756,286265617,0,66304,458752,0,2003859319,2828843402
1,5,"""b1c3""",1227,1190,1208.5,1107710756,286265617,0,66304,458752,8388608,2003859319,2694625674


In [3]:
min_move_idx = 3
df = df.filter(pl.col("move_idx") > min_move_idx)
df.head().collect()

id,move_idx,move_uci,white_elo,black_elo,avg_elo,state0,state1,state2,state3,state4,state5,state6,state7
i64,i64,str,i64,i64,f64,u32,u32,u32,u32,u32,u32,u32,u32
1,4,"""g8f6""",1227,1190,1208.5,1107710756,286265617,0,66304,458752,0,2003859319,2828843402
1,5,"""b1c3""",1227,1190,1208.5,1107710756,286265617,0,66304,458752,8388608,2003859319,2694625674
1,6,"""a7a6""",1227,1190,1208.5,1107710724,286265617,512,66304,458752,8388608,2003859319,2694625674
1,7,"""g1f3""",1227,1190,1208.5,1107710724,286265617,512,66304,458752,8388615,2003859312,2694625674
1,8,"""b7b5""",1227,1190,1208.5,1074156292,286265617,2097664,66304,458752,8388615,2003859312,2694625674


In [4]:
def extract_piece(colname: str, col: int) -> pl.Expr:
    return (pl.col(colname) // (2 ** (col * 4))) & 0xF

df = df.with_columns(
    *[
        extract_piece(f"state{i}", j).cast(pl.Float32).alias(f"tile{i}{j}")
        for i in range(8)
        for j in range(8)
    ]
).select(
    pl.exclude(*[f"state{i}" for i in range(8)]),
)
df.head().collect()

id,move_idx,move_uci,white_elo,black_elo,avg_elo,tile00,tile01,tile02,tile03,tile04,tile05,tile06,tile07,tile10,tile11,tile12,tile13,tile14,tile15,tile16,tile17,tile20,tile21,tile22,tile23,tile24,tile25,tile26,tile27,tile30,tile31,tile32,tile33,tile34,tile35,tile36,tile37,tile40,tile41,tile42,tile43,tile44,tile45,tile46,tile47,tile50,tile51,tile52,tile53,tile54,tile55,tile56,tile57,tile60,tile61,tile62,tile63,tile64,tile65,tile66,tile67,tile70,tile71,tile72,tile73,tile74,tile75,tile76,tile77
i64,i64,str,i64,i64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1,4,"""g8f6""",1227,1190,1208.5,4.0,2.0,3.0,5.0,6.0,0.0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,7.0,7.0,0.0,7.0,7.0,7.0,10.0,8.0,9.0,11.0,12.0,9.0,8.0,10.0
1,5,"""b1c3""",1227,1190,1208.5,4.0,2.0,3.0,5.0,6.0,0.0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,7.0,7.0,7.0,7.0,0.0,7.0,7.0,7.0,10.0,8.0,9.0,11.0,12.0,9.0,0.0,10.0
1,6,"""a7a6""",1227,1190,1208.5,4.0,0.0,3.0,5.0,6.0,0.0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,7.0,7.0,7.0,7.0,0.0,7.0,7.0,7.0,10.0,8.0,9.0,11.0,12.0,9.0,0.0,10.0
1,7,"""g1f3""",1227,1190,1208.5,4.0,0.0,3.0,5.0,6.0,0.0,2.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,7.0,7.0,7.0,0.0,7.0,7.0,7.0,10.0,8.0,9.0,11.0,12.0,9.0,0.0,10.0
1,8,"""b7b5""",1227,1190,1208.5,4.0,0.0,3.0,5.0,6.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,7.0,7.0,7.0,0.0,7.0,7.0,7.0,10.0,8.0,9.0,11.0,12.0,9.0,0.0,10.0


In [5]:
df_sampled = (
    df.group_by("id")
      .agg(pl.all().exclude("id").sample(fraction=fraction, seed=seed))
      .explode(pl.all().exclude("id"))
).sort("id", "move_idx")

df_sampled.head().collect()

id,move_idx,move_uci,white_elo,black_elo,avg_elo,tile00,tile01,tile02,tile03,tile04,tile05,tile06,tile07,tile10,tile11,tile12,tile13,tile14,tile15,tile16,tile17,tile20,tile21,tile22,tile23,tile24,tile25,tile26,tile27,tile30,tile31,tile32,tile33,tile34,tile35,tile36,tile37,tile40,tile41,tile42,tile43,tile44,tile45,tile46,tile47,tile50,tile51,tile52,tile53,tile54,tile55,tile56,tile57,tile60,tile61,tile62,tile63,tile64,tile65,tile66,tile67,tile70,tile71,tile72,tile73,tile74,tile75,tile76,tile77
i64,i64,str,i64,i64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1,18,"""f6e4""",1227,1190,1208.5,4.0,0.0,3.0,5.0,6.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,9.0,0.0,0.0,0.0,2.0,0.0,7.0,0.0,7.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,7.0,11.0,0.0,7.0,7.0,10.0,8.0,9.0,0.0,12.0,10.0,0.0,0.0
1,38,"""f5g4""",1227,1190,1208.5,4.0,0.0,3.0,5.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,6.0,0.0,1.0,1.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,7.0,8.0,0.0,0.0,9.0,0.0,0.0,7.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,10.0,0.0,0.0,0.0,12.0,10.0,0.0,0.0
1,41,"""e1d1""",1227,1190,1208.5,4.0,0.0,3.0,9.0,6.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,0.0,7.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,10.0,0.0,0.0,0.0,12.0,10.0,0.0,0.0
1,43,"""c3e4""",1227,1190,1208.5,4.0,0.0,3.0,6.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,10.0,0.0,0.0,0.0,12.0,10.0,0.0,0.0
1,51,"""h1e1""",1227,1190,1208.5,4.0,0.0,0.0,6.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,10.0,12.0,0.0,0.0,0.0


In [7]:
df_sampled.collect().write_parquet("output/per_move_sampled.parquet")